# Widzenie maszynowe
## Laboratorium 3 - Image skeletonization
*Autor: Paweł Mendroch* - [Github](https://github.com/FrozenTear7/computer-vision-lab/tree/master/lab3)

Wykorzystując kod ze strony z [tutorialem](https://scikit-image.org/docs/dev/auto_examples/edges/plot_skeleton.html), generuję szkielety dwoma metodami dla każdego obrazu z folderu `input`.

Kod uruchomieniowy jest zawarty w `run.sh`, iteruję po tablicy ścieżek do plików do przetworzenia, które zapisywane są w folderze `output`.